In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [8]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
from pymongo import MongoClient

In [4]:
db = MongoClient()

In [5]:
row = next(db.datasets.elementy.find().limit(1))

In [6]:
row.keys()

dict_keys(['_id', 'markdown', 'title', 'url', 'modalities', 'summary'])

In [153]:
db.datasets.habrahabr.drop_index("markdown_text")

OperationFailure: index not found with name [markdown_text]

In [155]:
db.datasets.postnauka.drop_index("markdown_text")

In [156]:
%%time

db.datasets.postnauka.create_index([("markdown", "text")], default_language='russian')

CPU times: user 3.24 ms, sys: 5.88 ms, total: 9.11 ms
Wall time: 8.54 s


'markdown_text'

In [157]:
%%time

db.datasets.habrahabr.create_index([("markdown", "text")], default_language='russian')

CPU times: user 52.5 ms, sys: 41.3 ms, total: 93.8 ms
Wall time: 4min 14s


'markdown_text'

In [127]:
for row in db.datasets.postnauka.find({"authors_names": {"$elemMatch": {"$eq": "Константин Воронцов"}}}):
    print(row)

Математические методы прогнозирования объемов продаж


In [137]:
%%time

for row in db.datasets.postnauka.find({'$text': {'$search': 'анализы данных'}},
                           { "score": { "$meta": "textScore" }}).sort([('score', {'$meta': 'textScore'})]).limit(5):
    print(row["_id"], row["title"], row["score"])

pn_1709 Анализ данных как область знания 2.062361139347119
pn_2537 Анализ данных 2.0473438112134357
pn_1887 FAQ: Анализ данных 2.0436088319431223
pn_681 Главы | Контроль 2.030854885057471
pn_658 Анализ социальных сетей в интернете 2.0223983893293065
CPU times: user 8.07 ms, sys: 4.04 ms, total: 12.1 ms
Wall time: 69.6 ms


In [148]:
db.datasets.habrahabr.name

'habrahabr'

In [160]:
results = db.datasets.postnauka.find({'$text': {'$search': 'анализы данных'}},
                           { "score": { "$meta": "textScore" }}) \
            .limit(5) \
            .sort([('score', {'$meta': 'textScore'})])

In [159]:
for collection in [db.datasets[col] for col in ["habrahabr", "postnauka"]]:
    if not ("markdown_text" in collection.index_information().keys()):
        collection.create_index([("markdown", "text")], default_language='russian')
        print("%s"%collection.name)

In [71]:
%%time

if not ("markdown_text" in db.model.all_docs.index_information().keys()):
    db.model.all_docs.create_index([("markdown", "text")], default_language='russian')
    print("done")

done
CPU times: user 663 ms, sys: 2.3 s, total: 2.96 s
Wall time: 14.3 s


In [9]:
from server.artm_lib import ArtmDataSource, ArtmBridge, ArtmModel

In [ ]:
ads = ArtmDataSource()

In [ ]:
model = ArtmModel("../server/hartm")

In [101]:
%%time

res = ads.search_query_in_models_docs(query = "биология", limit=100)
model.get_topics_by_docs_ids(res)

CPU times: user 156 ms, sys: 66.1 ms, total: 223 ms
Wall time: 221 ms


In [12]:
theta, thresholds, topics, tid_lid = model.get_topics_by_doc_id("azaza")

/Users/aksholokhov/Local Docs/repos/rysearch/experiments/server/artm_lib.py:79: DeprecationWarning: invalid escape sequence \d
  if regex.match("^topic_\d+$", tid):


In [16]:
topics_for_docs = []

for doc in res:
    if doc["doc_id"] not in thresholds.index:
        continue
    topics_for_doc = {}
    comparsion = theta[doc["doc_id"]] > thresholds[doc["doc_id"]]
    topics_for_doc["doc_id"] = doc["doc_id"]
    last_level_topics = list(comparsion[comparsion == True].index)
    levels_count = tid_lid[last_level_topics[0]][0] + 1
    topics_for_doc["level_%d"%(levels_count-1)] = last_level_topics  
    for lid in range(1, levels_count)[::-1]:
        current_level = topics_for_doc["level_%d"%(lid)]
        higher_level = []
        for topic in current_level:
            higher_level += topics[topic]["parents"]
        topics_for_doc["level_%d"%(lid-1)] = higher_level
    topics_for_docs.append(topics_for_doc)

In [152]:
res = bridge.data_source.search_query_in_models_docs("машинное обучение")

theta = bridge.model.theta
doc_theta = bridge.model._doc_theta
thresholds = bridge.model._doc_thresholds
topics = bridge.model.topics
tid_lid = bridge.model._to_lid_tid_map


import pandas as pd 
import numpy as np

lowest_level_counter = pd.Series(np.zeros(len(doc_theta.index)), index = doc_theta.index)

for doc in res:
    if doc["doc_id"] not in thresholds.index:
        continue
    topics_for_doc = {}
    lowest_level_counter += (doc_theta[doc["doc_id"]] > thresholds[doc["doc_id"]]).map(lambda x: 1 if x else 0)

levels_count = tid_lid[lowest_level_counter.index[0]][0] + 1
    
answer = pd.Series(np.zeros(len(theta.index)), index = theta.index)
answer[lowest_level_counter.index] = lowest_level_counter

for lid in range(0, levels_count-1)[::-1]:
    curr_level_topics = list(filter(lambda x: x.startswith("level_%d_t"%(lid)), answer.index))
    for topic in curr_level_topics:
        for child in topics[topic]["children"]:
            answer[topic] += answer[child]

for lid in range(0, levels_count)[::-1]:
    curr_level_topics = list(filter(lambda x: x.startswith("level_%d_t"%(lid)), answer.index))
    answer[curr_level_topics] /= sum(answer[curr_level_topics])

dict(answer)

{'level_0_background_0': 0.0,
 'level_0_topic_0': 0.0,
 'level_0_topic_1': 0.0,
 'level_0_topic_10': 0.0,
 'level_0_topic_11': 0.0,
 'level_0_topic_12': 0.0,
 'level_0_topic_13': 0.0,
 'level_0_topic_14': 0.0,
 'level_0_topic_15': 0.16666666666666666,
 'level_0_topic_16': 0.0,
 'level_0_topic_17': 0.0,
 'level_0_topic_18': 0.66666666666666663,
 'level_0_topic_2': 0.0,
 'level_0_topic_3': 0.0,
 'level_0_topic_4': 0.16666666666666666,
 'level_0_topic_5': 0.0,
 'level_0_topic_6': 0.0,
 'level_0_topic_7': 0.0,
 'level_0_topic_8': 0.0,
 'level_0_topic_9': 0.0,
 'level_1_topic_0': 0.0,
 'level_1_topic_1': 0.0,
 'level_1_topic_10': 0.0,
 'level_1_topic_11': 0.0,
 'level_1_topic_12': 0.0,
 'level_1_topic_13': 0.0,
 'level_1_topic_14': 0.0,
 'level_1_topic_15': 0.0,
 'level_1_topic_16': 0.0,
 'level_1_topic_17': 0.0,
 'level_1_topic_18': 0.0,
 'level_1_topic_19': 0.0,
 'level_1_topic_2': 0.0,
 'level_1_topic_20': 0.0,
 'level_1_topic_21': 0.0,
 'level_1_topic_22': 0.0,
 'level_1_topic_23': 0.18

In [135]:
lowest_level_counter

level_1_topic_0     0.0
level_1_topic_1     0.0
level_1_topic_2     0.0
level_1_topic_3     0.0
level_1_topic_4     0.0
level_1_topic_5     0.0
level_1_topic_6     0.0
level_1_topic_7     0.0
level_1_topic_8     4.0
level_1_topic_9     0.0
level_1_topic_10    0.0
level_1_topic_11    0.0
level_1_topic_12    0.0
level_1_topic_13    0.0
level_1_topic_14    0.0
level_1_topic_15    0.0
level_1_topic_16    0.0
level_1_topic_17    0.0
level_1_topic_18    0.0
level_1_topic_19    0.0
level_1_topic_20    0.0
level_1_topic_21    0.0
level_1_topic_22    0.0
level_1_topic_23    3.0
level_1_topic_24    6.0
level_1_topic_25    0.0
level_1_topic_26    0.0
level_1_topic_27    0.0
level_1_topic_28    0.0
level_1_topic_29    0.0
level_1_topic_30    0.0
level_1_topic_31    0.0
level_1_topic_32    0.0
level_1_topic_33    0.0
level_1_topic_34    0.0
level_1_topic_35    0.0
level_1_topic_36    0.0
level_1_topic_37    0.0
level_1_topic_38    1.0
level_1_topic_39    0.0
level_1_topic_40    0.0
level_1_topic_41

In [18]:
for doc in topics_for_docs:
    level_0_topics = []
    for topic in doc["level_1_topics"]:
        level_0_topics += topics[topic]["parents"]
    doc["level_0_topics"] = level_0_topics

In [24]:
topics["level_0_topic_3"]

{'children': ['level_1_topic_3'],
 'level_id': 0,
 'parents': [],
 'top_words': ['культура', 'литература', 'культурология'],
 'weight': 361}

In [11]:
%%time

bridge = ArtmBridge("../server/hartm")

CPU times: user 2min 52s, sys: 8.26 s, total: 3min 1s
Wall time: 2min 50s


In [26]:
%%time
bridge.search_documents("анализ данных", limit=100)

CPU times: user 128 ms, sys: 11 ms, total: 139 ms
Wall time: 280 ms


{'level_0_background_0': 0.0,
 'level_0_topic_0': 0.012345679012345678,
 'level_0_topic_1': 0.12345679012345678,
 'level_0_topic_10': 0.080246913580246909,
 'level_0_topic_11': 0.018518518518518517,
 'level_0_topic_12': 0.10493827160493827,
 'level_0_topic_13': 0.055555555555555552,
 'level_0_topic_14': 0.0,
 'level_0_topic_15': 0.067901234567901231,
 'level_0_topic_16': 0.024691358024691357,
 'level_0_topic_17': 0.055555555555555552,
 'level_0_topic_18': 0.13580246913580246,
 'level_0_topic_2': 0.061728395061728392,
 'level_0_topic_3': 0.030864197530864196,
 'level_0_topic_4': 0.0,
 'level_0_topic_5': 0.067901234567901231,
 'level_0_topic_6': 0.018518518518518517,
 'level_0_topic_7': 0.043209876543209874,
 'level_0_topic_8': 0.055555555555555552,
 'level_0_topic_9': 0.043209876543209874,
 'level_1_topic_0': 0.0,
 'level_1_topic_1': 0.035460992907801421,
 'level_1_topic_10': 0.046099290780141841,
 'level_1_topic_11': 0.0035460992907801418,
 'level_1_topic_12': 0.060283687943262408,
 'l

## Мержинг коллекций 

In [13]:
%%time

all_docs_ids = set(bridge._model._doc_theta.columns)

for collection in db.datasets.collection_names():
    for doc in db.datasets[collection].find():
        if doc["_id"] in all_docs_ids:
            db.model.all_docs.insert_one(doc)

CPU times: user 35 s, sys: 3.15 s, total: 38.1 s
Wall time: 44.9 s


In [14]:
db.model.all_docs.create_index([("markdown", "text")], default_language="russian")

'markdown_text'

In [15]:
db.model.all_docs.count()

6234